<h2>Создание датасета</h2>

In [63]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [64]:
df = pd.read_csv('sale_tasks_dataset.csv', decimal='.')

In [65]:
df['finish_dttm'] = pd.to_datetime(df['finish_dttm'], format='%Y-%m-%d %H:%M:%S')
df['day_of_week'] = df['finish_dttm'].dt.dayofweek
df['is_holiday'] = df.day_of_week.apply(lambda x: 1 if x > 4 else 0) # создание is_holiday

df.drop_duplicates(subset=['customer_id'], inplace=True, keep='first') # дубликаты клиентов

df.dropna(subset=['age', 'gender_cd', 'state_talk_time_sec'], inplace=True) # пустые значения по age, gender_cd

df['age'] = pd.to_numeric(df['age'], errors='coerce')
df = df[(df['age'] <= 65) & (df['age'] >= 18)] # очистка по age
IQR = df["state_talk_time_sec"].quantile(0.75) - df["state_talk_time_sec"].quantile(0.25)
high = df["state_talk_time_sec"].quantile(0.75) + IQR * 3
df = df[df["state_talk_time_sec"] < high] # чистка длительности звонка


df['gender_cd'] = df['gender_cd'].map({'F': 0, 'M': 1})

df['region_size'].fillna('No', inplace=True)
mapping = {
    'No': 0,
    'town': 1,
    'urban': 2,
    'msk': 3,
    'rural': 4
}

df['region_size_numeric'] = df['region_size'].map(mapping)

df["approval_flag"] = df["wo_hit_status_result_desc"].replace({"Дозвон, Отказ": 0, "Дозвон, Успешно": 1})

In [66]:
df.to_csv('data_preprocessed.csv', index=False)

<h3>Дополнительно</h3>

In [67]:
df.to_csv('data_preprocessed_sort.csv', columns=['age', 'gender_cd', 'day_of_week', 'is_holiday', 'region_size_numeric'] , index=False)